### Text Classification

The goal of this notebook is to walk through the machine learning step of the text classification process.

1) Encoding

2) Partitioning the dataset into distinct subgroups

3) Vectorization (Term Frequency Inverse Document Frequency (TF-IDF))


In [ ]:
%matplotlib widget
import glob
import sys
sys.path.append('/Users/nmiles/PACMan_dist/')


from astropy.visualization import ImageNormalize, LinearStretch, ZScaleInterval
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import pandas as pd
from utils.tokenizer import PACManTokenizer
import pacman2020



from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix



In [ ]:
def read_category_label(fname):
    flabel = fname.replace('_training.txt','_Scientific_Category.txt')
    with open(flabel, 'r') as fobj:
        lines = fobj.readlines()
    print(lines)

In [ ]:
cy22 = '/Users/nmiles/PACMan_dist/proposal_data/Cy22_Proposals_txt/'
cy23 = '/Users/nmiles/PACMan_dist/proposal_data/Cy23_Proposals_txt'
cy24 = '/Users/nmiles/PACMan_dist/proposal_data/Cy24_Proposals_txt'
cy25 = '/Users/nmiles/PACMan_dist/proposal_data/Cy25_Proposals_txt'

In [ ]:
fname = '/Users/nmiles/PACMan_dist/proposal_data/Cy25_proposals_txt/training_corpus/0001_training.txt'

In [ ]:
read_category_label(fname)

In [ ]:
pacman = PACManTokenizer()
pacman.get_stop_words(
        fname='/Users/nmiles/PACMan_dist/libs/stopwords.txt'
    )

In [ ]:
text, cleaned_text, tokens = pacman.run_tokenization(fname=fname, N=20, plot=True)

In [ ]:
print(text[:650])

In [ ]:
print(cleaned_text[:500])

In [ ]:
flist_text = glob.glob(f"{cy25}/training_corpus/*training.txt")
flist_label = glob.glob(f"{cy25}/training_corpus/*_Scientific_Category.txt")

In [ ]:
train_df, data = pacman2020.read_in_dataset(flist=flist_text, parallel=False)

In [ ]:
train_df.head()

In [ ]:
categories = train_df['category'].value_counts()
fig, ax = plt.subplots(nrows=1, ncols=1)
categories.plot.barh(ax=ax)

In [ ]:
def create_balanced_subset(df, categories=[]):
    subsets = {}
    for category in categories:
        data = df[df['category'] == category].iloc[:150,:]
        subsets[category] = data
    return subsets

In [ ]:
train_df.info()

In [ ]:
subsets = create_balanced_subset(train_df, categories=np.unique(train_df['category']))

In [ ]:
[print(len(subsets[key])) for key in subsets.keys()]

In [ ]:
train_df['category'].factorize()

In [ ]:
train_df['category_id'] = train_df['category'].factorize()[0]

In [ ]:
train_df.tail()

In [ ]:
category_id_df_train = train_df[['category','category_id']]
category_to_id_train = dict(category_id_df_train.values)
id_to_category_train = dict(category_id_df_train[['category_id', 'category']].values)


In [ ]:
id_to_category_test[0] = 'stellar populations and the ism'
id_to_category_test[1] = id_to_category_test[1].lower() 
id_to_category_test[2] = id_to_category_test[2].lower()
id_to_category_test[3] = 'planets and planet formation'
id_to_category_test[4] = 'galaxies and the igm'
id_to_category_test[5] = 'large scale structure of the universe'
id_to_category_test[6] = 'supermassive black holes and active galaxies'

In [ ]:
id_to_category_train

In [ ]:
tfidf_vect = TfidfVectorizer(max_features=10000,
#                              tokenizer=pacman2020.spacy_tokenizer,
#                              analyzer='word',
                             stop_words='english',
                             use_idf=True,
                             norm='l2',
                             ngram_range=(1, 2))

In [ ]:
count_vect = CountVectorizer(max_features=10000, tokenizer=pacman2020.spacy_tokenizer)

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(train_df['text'], train_df['category_id'], test_size=0.2, train_size=0.8)

In [ ]:
count_vect = count_vect.fit(x_train)

In [ ]:
tfidf_vectorizer_vectors=tfidf_vect.fit_transform(x_train)

In [ ]:
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]
 
# place tf-idf values in a pandas data frame
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vect.get_feature_names(), columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

In [ ]:
Encoder = LabelEncoder()
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

In [ ]:
nb_tfidf = Pipeline([('vect', tfidf_vect),
               ('clf', MultinomialNB(alpha=0.05)),
              ])


In [ ]:
nb_count = Pipeline([('vect', count_vect),
               ('clf', MultinomialNB()),
              ])

In [ ]:
nb_tfidf.fit(train_df['text'], train_df['category_id'])

In [ ]:
nb_count.fit(train_df['text'], train_df['category_id'])

In [ ]:
flist_text_test = glob.glob(f"{cy24}/training_corpus/*training.txt")
flist_label_test = glob.glob(f"{cy24}/training_corpus/*_Scientific_Category.txt")
test_df, data = pacman2020.read_in_dataset(flist_text=flist_text_test, flist_label=flist_label_test, notebook=True)

In [ ]:
test_df['category'].factorize()

In [ ]:
test_df['category_id'] = test_df['category'].factorize()[0]
category_id_df_test = test_df[['category','category_id']]
category_to_id_test = dict(category_id_df_test.values)
id_to_category_test = dict(category_id_df_test[['category_id', 'category']].values)

In [ ]:
id_to_category_test

In [ ]:
id_to_category_test[0] = 'stellar populations and the ism'
id_to_category_test[1] = id_to_category_test[1].lower() 
id_to_category_test[2] = id_to_category_test[2].lower()
id_to_category_test[3] = 'planets and planet formation'
id_to_category_test[4] = 'galaxies and the igm'
id_to_category_test[5] = 'large scale structure of the universe'
id_to_category_test[6] = 'supermassive black holes and active galaxies'

In [ ]:
predictions = nb_tfidf.predict(test_df['text'])

In [ ]:
accuracy_score(test_df['category_id'], predictions)

In [ ]:
predictions_count = nb_count.predict(test_df['text'])

In [ ]:
accuracy_score(test_df['category_id'], predictions_count)

In [ ]:
confusion_mat = confusion_matrix(test_df['category_id'], predictions)

In [ ]:
confusion_mat_count = confusion_matrix(test_df['category_id'], predictions_count)

In [ ]:
print(confusion_mat_count)

In [ ]:
print(classification_report(test_df['category_id'], predictions_count , target_names=list(id_to_category_test.values())))

In [ ]:
print(classification_report(test_df['category_id'], predictions , target_names=list(id_to_category_test.values())))

### Cycle 25 testing using the UAT categories

In [ ]:
proposal_classifications = pd.read_csv('/Users/nmiles/PACMan_dist/cycle_25_classifications.txt')

Parse the filenames to get the proposal number

In [ ]:
proposal_numbers = [int(val.split('/')[-1].split('_')[0]) for val in flist_text]
flist_num = list(zip(flist_text, proposal_numbers))
flist_num.sort(key=lambda val: val[1])
flist_sorted, proposal_num = list(zip(*flist_num))

In [ ]:
hand_classified_null = proposal_classifications[proposal_classifications['classification'].isnull()]

In [ ]:
proposal_classifications.info()

In [ ]:
a = np.ediff1d(proposal_num)
idx = list(map(int, np.where(a>1)[0]))
missing_proposals = [proposal_num[val]+1 for val in idx]

In [ ]:
missing_proposals

In [ ]:
hand_classified_null

In [ ]:
proposal_classifications['fname'] = [np.nan]*len(proposal_classifications)

In [ ]:
proposal_classifications.head()

In [ ]:
for num, fname in zip(proposal_num, flist_sorted, ):
    proposal_classifications['fname'].loc[num-1] = fname

In [ ]:
proposal_classifications['classification'].factorize()

In [ ]:
df, data = pacman2020.read_in_dataset(flist_label=flist_label, flist_text=flist_sorted)